In [1]:
%run ../paths.py
CONST = CONST()

In [2]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
tqdm.pandas()
import ipywidgets
import warnings
warnings.filterwarnings('ignore')
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import metrics


from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from scipy.sparse import csr_matrix, hstack


/home/sandhya/Project/final/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
train = pd.read_csv("../dataset/cleaned_data")

In [4]:
train[train.columns[2:]].head()

,tweet,class,neg,pos,neu,compound,nouns,prons,verbs,adjs,...,num_symbols,stopword_count,emoji_count,hashtag,user,allcaps,total_length,num_words,num_unique_words,words_vs_unique
0,mann these hoes not loyal,1,0.390,0.000,0.610,-0.3724,3,0,1,0,...,0,0,0,0,0,0,25.0,5.0,5.0,1.0
1,fabulous weekend beautiful souls,0,0.000,0.549,0.451,0.8074,3,0,1,2,...,0,2,0,1,0,0,32.0,4.0,4.0,1.0
2,these hoes not loyal,1,0.299,0.000,0.701,-0.3724,4,0,1,1,...,0,1,0,0,1,0,20.0,4.0,4.0,1.0
3,newhair,0,0.000,0.000,1.000,0.0000,2,0,1,0,...,0,0,0,13,0,0,7.0,1.0,1.0,1.0
4,not trust these hoes,1,0.278,0.000,0.722,-0.4023,2,0,1,1,...,0,2,0,0,0,0,20.0,4.0,4.0,1.0


In [5]:
test = pd.read_csv("../dataset/test_cleaned_data")

In [6]:
test[test.columns[1:]].head()

,tweet,class,neg,pos,neu,compound,nouns,prons,verbs,adjs,...,num_symbols,stopword_count,emoji_count,hashtag,user,allcaps,total_length,num_words,num_unique_words,words_vs_unique
0,look ho look yes look no,1,0.000,0.505,0.495,0.8814,3,4,3,0,...,0,8,0,0,1,2,24.0,6.0,4.0,0.666667
1,lots some favorite not,0,0.000,0.223,0.777,0.5106,5,3,0,2,...,0,7,0,4,1,0,22.0,4.0,4.0,1.000000
2,wrong ppl nowadays was conducting sampling oth...,0,0.119,0.000,0.881,-0.4767,8,0,6,4,...,1,8,0,0,0,0,94.0,14.0,14.0,1.000000
3,lakers trash,1,0.000,0.000,1.000,0.0000,2,0,1,1,...,0,1,0,0,1,0,12.0,2.0,2.0,1.000000
4,will playing rf simpson nov,0,0.000,0.160,0.840,0.2714,5,0,2,1,...,0,4,0,2,1,0,27.0,5.0,5.0,1.000000


In [7]:
frames = [train[train.columns[2:]],test[test.columns[1:]]]
df = pd.concat(frames,ignore_index=True)


In [8]:
df.head()

,tweet,class,neg,pos,neu,compound,nouns,prons,verbs,adjs,...,num_symbols,stopword_count,emoji_count,hashtag,user,allcaps,total_length,num_words,num_unique_words,words_vs_unique
0,mann these hoes not loyal,1,0.390,0.000,0.610,-0.3724,3,0,1,0,...,0,0,0,0,0,0,25.0,5.0,5.0,1.0
1,fabulous weekend beautiful souls,0,0.000,0.549,0.451,0.8074,3,0,1,2,...,0,2,0,1,0,0,32.0,4.0,4.0,1.0
2,these hoes not loyal,1,0.299,0.000,0.701,-0.3724,4,0,1,1,...,0,1,0,0,1,0,20.0,4.0,4.0,1.0
3,newhair,0,0.000,0.000,1.000,0.0000,2,0,1,0,...,0,0,0,13,0,0,7.0,1.0,1.0,1.0
4,not trust these hoes,1,0.278,0.000,0.722,-0.4023,2,0,1,1,...,0,2,0,0,0,0,20.0,4.0,4.0,1.0


In [9]:
df.dropna(inplace=True)

In [10]:
drop_rows = train[train['total_length']<3]
train.drop(drop_rows.index, axis=0,inplace=True)


In [11]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41990 entries, 0 to 42018
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             41990 non-null  int64  
 1   Unnamed: 0.1           41990 non-null  int64  
 2   tweet                  41990 non-null  object 
 3   class                  41990 non-null  int64  
 4   neg                    41990 non-null  float64
 5   pos                    41990 non-null  float64
 6   neu                    41990 non-null  float64
 7   compound               41990 non-null  float64
 8   nouns                  41990 non-null  int64  
 9   prons                  41990 non-null  int64  
 10  verbs                  41990 non-null  int64  
 11  adjs                   41990 non-null  int64  
 12  advs                   41990 non-null  int64  
 13  num_exclamation_marks  41990 non-null  int64  
 14  num_question_marks     41990 non-null  int64  
 15  nu

# tweets as features and only wordvectorizer

In [12]:
%run ../src/features/vectorization

In [13]:
train_word_vec,test_word_vec = ngram_vectorize(TfidfVectorizer,train["tweet"],train["class"],test["tweet"],top_k=20000,NGRAM_RANGE=(1,2))

In [14]:
train_char_vec,test_char_vec = ngram_vectorize(TfidfVectorizer,train["tweet"],train["class"],test["tweet"],top_k=30000,TOKEN_MODE='char',NGRAM_RANGE=(2,6))

In [15]:
train_vec_combined = hstack((train_word_vec,train_char_vec)).tocsr()

In [16]:
test_vec_combined = hstack((test_word_vec,test_char_vec)).tocsr()

In [17]:
penalty = ['l1', 'l2']
C = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]
class_weight = [{1:0.5, 0:0.5}, {1:0.4, 0:0.6}, {1:0.6, 0:0.4}, {1:0.7, 0:0.3}]
solver = ['liblinear', 'saga']

param_grid = dict(penalty=penalty,
                  C=C,
                  class_weight=class_weight,
                  solver=solver)

grid_logistic = GridSearchCV(estimator=LogisticRegression(),
                    param_grid=param_grid,
                    scoring='f1',
                    verbose=2,cv=10,refit=True)
grid_logistic_result = grid_logistic.fit(train_word_vec, train["class"])

print('Best Score: ', grid_logistic_result.best_score_)
print('Best Params: ', grid_logistic_result.best_params_)


Fitting 10 folds for each of 128 candidates, totalling 1280 fits
[CV] C=0.0001, class_weight={1: 0.5, 0: 0.5}, penalty=l1, solver=liblinear 
[CV]  C=0.0001, class_weight={1: 0.5, 0: 0.5}, penalty=l1, solver=liblinear, total=   0.1s
[CV] C=0.0001, class_weight={1: 0.5, 0: 0.5}, penalty=l1, solver=liblinear 
[CV]  C=0.0001, class_weight={1: 0.5, 0: 0.5}, penalty=l1, solver=liblinear, total=   0.1s
[CV] C=0.0001, class_weight={1: 0.5, 0: 0.5}, penalty=l1, solver=liblinear 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


[CV]  C=0.0001, class_weight={1: 0.5, 0: 0.5}, penalty=l1, solver=liblinear, total=   0.1s
[CV] C=0.0001, class_weight={1: 0.5, 0: 0.5}, penalty=l1, solver=liblinear 
[CV]  C=0.0001, class_weight={1: 0.5, 0: 0.5}, penalty=l1, solver=liblinear, total=   0.1s
[CV] C=0.0001, class_weight={1: 0.5, 0: 0.5}, penalty=l1, solver=liblinear 
[CV]  C=0.0001, class_weight={1: 0.5, 0: 0.5}, penalty=l1, solver=liblinear, total=   0.1s
[CV] C=0.0001, class_weight={1: 0.5, 0: 0.5}, penalty=l1, solver=liblinear 
[CV]  C=0.0001, class_weight={1: 0.5, 0: 0.5}, penalty=l1, solver=liblinear, total=   0.0s
[CV] C=0.0001, class_weight={1: 0.5, 0: 0.5}, penalty=l1, solver=liblinear 
[CV]  C=0.0001, class_weight={1: 0.5, 0: 0.5}, penalty=l1, solver=liblinear, total=   0.1s
[CV] C=0.0001, class_weight={1: 0.5, 0: 0.5}, penalty=l1, solver=liblinear 
[CV]  C=0.0001, class_weight={1: 0.5, 0: 0.5}, penalty=l1, solver=liblinear, total=   0.1s
[CV] C=0.0001, class_weight={1: 0.5, 0: 0.5}, penalty=l1, solver=liblinear

[CV]  C=0.0001, class_weight={1: 0.4, 0: 0.6}, penalty=l1, solver=saga, total=   0.0s
[CV] C=0.0001, class_weight={1: 0.4, 0: 0.6}, penalty=l1, solver=saga 
[CV]  C=0.0001, class_weight={1: 0.4, 0: 0.6}, penalty=l1, solver=saga, total=   0.1s
[CV] C=0.0001, class_weight={1: 0.4, 0: 0.6}, penalty=l1, solver=saga 
[CV]  C=0.0001, class_weight={1: 0.4, 0: 0.6}, penalty=l1, solver=saga, total=   0.0s
[CV] C=0.0001, class_weight={1: 0.4, 0: 0.6}, penalty=l1, solver=saga 
[CV]  C=0.0001, class_weight={1: 0.4, 0: 0.6}, penalty=l1, solver=saga, total=   0.1s
[CV] C=0.0001, class_weight={1: 0.4, 0: 0.6}, penalty=l1, solver=saga 
[CV]  C=0.0001, class_weight={1: 0.4, 0: 0.6}, penalty=l1, solver=saga, total=   0.1s
[CV] C=0.0001, class_weight={1: 0.4, 0: 0.6}, penalty=l2, solver=liblinear 
[CV]  C=0.0001, class_weight={1: 0.4, 0: 0.6}, penalty=l2, solver=liblinear, total=   0.1s
[CV] C=0.0001, class_weight={1: 0.4, 0: 0.6}, penalty=l2, solver=liblinear 
[CV]  C=0.0001, class_weight={1: 0.4, 0: 0.

[CV]  C=0.0001, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=liblinear, total=   0.1s
[CV] C=0.0001, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=saga 
[CV]  C=0.0001, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=saga, total=   2.0s
[CV] C=0.0001, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=saga 
[CV]  C=0.0001, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=saga, total=   2.1s
[CV] C=0.0001, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=saga 
[CV]  C=0.0001, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=saga, total=   2.2s
[CV] C=0.0001, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=saga 
[CV]  C=0.0001, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=saga, total=   2.2s
[CV] C=0.0001, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=saga 
[CV]  C=0.0001, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=saga, total=   2.0s
[CV] C=0.0001, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=saga 
[CV]  C=0.0001, class_weight={1: 0.6, 0: 0.4}, penalt

...

[CV]  C=1000, class_weight={1: 0.4, 0: 0.6}, penalty=l1, solver=liblinear, total=   0.8s
[CV] C=1000, class_weight={1: 0.4, 0: 0.6}, penalty=l1, solver=liblinear 
[CV]  C=1000, class_weight={1: 0.4, 0: 0.6}, penalty=l1, solver=liblinear, total=   0.9s
[CV] C=1000, class_weight={1: 0.4, 0: 0.6}, penalty=l1, solver=liblinear 
[CV]  C=1000, class_weight={1: 0.4, 0: 0.6}, penalty=l1, solver=liblinear, total=   1.0s
[CV] C=1000, class_weight={1: 0.4, 0: 0.6}, penalty=l1, solver=liblinear 
[CV]  C=1000, class_weight={1: 0.4, 0: 0.6}, penalty=l1, solver=liblinear, total=   0.9s
[CV] C=1000, class_weight={1: 0.4, 0: 0.6}, penalty=l1, solver=liblinear 
[CV]  C=1000, class_weight={1: 0.4, 0: 0.6}, penalty=l1, solver=liblinear, total=   0.9s
[CV] C=1000, class_weight={1: 0.4, 0: 0.6}, penalty=l1, solver=saga ..
[CV]  C=1000, class_weight={1: 0.4, 0: 0.6}, penalty=l1, solver=saga, total= 8.3min
[CV] C=1000, class_weight={1: 0.4, 0: 0.6}, penalty=l1, solver=saga ..
[CV]  C=1000, class_weight={1: 0.

[CV]  C=1000, class_weight={1: 0.6, 0: 0.4}, penalty=l1, solver=saga, total=11.0min
[CV] C=1000, class_weight={1: 0.6, 0: 0.4}, penalty=l1, solver=saga ..
[CV]  C=1000, class_weight={1: 0.6, 0: 0.4}, penalty=l1, solver=saga, total= 7.8min
[CV] C=1000, class_weight={1: 0.6, 0: 0.4}, penalty=l1, solver=saga ..
[CV]  C=1000, class_weight={1: 0.6, 0: 0.4}, penalty=l1, solver=saga, total= 7.8min
[CV] C=1000, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=liblinear 
[CV]  C=1000, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=liblinear, total=   1.0s
[CV] C=1000, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=liblinear 
[CV]  C=1000, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=liblinear, total=   0.6s
[CV] C=1000, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=liblinear 
[CV]  C=1000, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=liblinear, total=   0.9s
[CV] C=1000, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=liblinear 
[CV]  C=1000, class_weight={1: 0.6, 0: 0.4}

[CV]  C=1000, class_weight={1: 0.7, 0: 0.3}, penalty=l2, solver=liblinear, total=   0.9s
[CV] C=1000, class_weight={1: 0.7, 0: 0.3}, penalty=l2, solver=saga ..
[CV]  C=1000, class_weight={1: 0.7, 0: 0.3}, penalty=l2, solver=saga, total=   2.0s
[CV] C=1000, class_weight={1: 0.7, 0: 0.3}, penalty=l2, solver=saga ..
[CV]  C=1000, class_weight={1: 0.7, 0: 0.3}, penalty=l2, solver=saga, total=   1.8s
[CV] C=1000, class_weight={1: 0.7, 0: 0.3}, penalty=l2, solver=saga ..
[CV]  C=1000, class_weight={1: 0.7, 0: 0.3}, penalty=l2, solver=saga, total=   2.0s
[CV] C=1000, class_weight={1: 0.7, 0: 0.3}, penalty=l2, solver=saga ..
[CV]  C=1000, class_weight={1: 0.7, 0: 0.3}, penalty=l2, solver=saga, total=   1.9s
[CV] C=1000, class_weight={1: 0.7, 0: 0.3}, penalty=l2, solver=saga ..
[CV]  C=1000, class_weight={1: 0.7, 0: 0.3}, penalty=l2, solver=saga, total=   1.9s
[CV] C=1000, class_weight={1: 0.7, 0: 0.3}, penalty=l2, solver=saga ..
[CV]  C=1000, class_weight={1: 0.7, 0: 0.3}, penalty=l2, solver=s

[Parallel(n_jobs=1)]: Done 1280 out of 1280 | elapsed: 660.5min finished


Best Score:  0.9286056813756783
Best Params:  {'C': 10, 'class_weight': {1: 0.6, 0: 0.4}, 'penalty': 'l2', 'solver': 'saga'}


In [20]:
model = LogisticRegression().fit(train_word_vec, train["class"])

In [23]:
#increses from 92.13 to 92.86
y = model.predict(test_word_vec)
metrics.f1_score(test['class'],y)

0.9213506933418015

In [32]:
y = grid_logistic_result.predict(train_word_vec)
print("training f1 ",metrics.f1_score(train['class'],y)*100)
print("average f1 by cv 92.86056813756783")
y = grid_logistic_result.predict(test_word_vec)
print("testing f1",metrics.f1_score(test['class'],y)*100)


training f1  96.77105398751932
average f1 by cv 92.86056813756783
testing f1 93.22643642717648


# LR with combined vectors

In [33]:
penalty = ['l1', 'l2']
C = [0.00001,0.0001, 0.001, 0.01, 0.1, 1, 10]
class_weight = [{1:0.4, 0:0.6}, {1:0.6, 0:0.4}, {1:0.7, 0:0.3}]
solver = ['liblinear', 'saga']

param_grid = dict(penalty=penalty,
                  C=C,
                  class_weight=class_weight,
                  solver=solver)

grid_logistic_combined = GridSearchCV(estimator=LogisticRegression(),
                    param_grid=param_grid,
                    scoring='f1',
                    verbose=2,cv=10,refit=True)
grid_logistic_combined_result = grid_logistic_combined.fit(train_vec_combined, train["class"])

print('Best Score: ', grid_logistic_combined_result.best_score_)
print('Best Params: ', grid_logistic_combined_result.best_params_)


Fitting 10 folds for each of 84 candidates, totalling 840 fits
[CV] C=1e-05, class_weight={1: 0.4, 0: 0.6}, penalty=l1, solver=liblinear 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  C=1e-05, class_weight={1: 0.4, 0: 0.6}, penalty=l1, solver=liblinear, total=   0.5s
[CV] C=1e-05, class_weight={1: 0.4, 0: 0.6}, penalty=l1, solver=liblinear 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s


[CV]  C=1e-05, class_weight={1: 0.4, 0: 0.6}, penalty=l1, solver=liblinear, total=   0.5s
[CV] C=1e-05, class_weight={1: 0.4, 0: 0.6}, penalty=l1, solver=liblinear 
[CV]  C=1e-05, class_weight={1: 0.4, 0: 0.6}, penalty=l1, solver=liblinear, total=   0.6s
[CV] C=1e-05, class_weight={1: 0.4, 0: 0.6}, penalty=l1, solver=liblinear 
[CV]  C=1e-05, class_weight={1: 0.4, 0: 0.6}, penalty=l1, solver=liblinear, total=   0.7s
[CV] C=1e-05, class_weight={1: 0.4, 0: 0.6}, penalty=l1, solver=liblinear 
[CV]  C=1e-05, class_weight={1: 0.4, 0: 0.6}, penalty=l1, solver=liblinear, total=   0.7s
[CV] C=1e-05, class_weight={1: 0.4, 0: 0.6}, penalty=l1, solver=liblinear 
[CV]  C=1e-05, class_weight={1: 0.4, 0: 0.6}, penalty=l1, solver=liblinear, total=   0.5s
[CV] C=1e-05, class_weight={1: 0.4, 0: 0.6}, penalty=l1, solver=liblinear 
[CV]  C=1e-05, class_weight={1: 0.4, 0: 0.6}, penalty=l1, solver=liblinear, total=   0.3s
[CV] C=1e-05, class_weight={1: 0.4, 0: 0.6}, penalty=l1, solver=liblinear 
[CV]  C=1e

...

[CV]  C=10, class_weight={1: 0.4, 0: 0.6}, penalty=l1, solver=liblinear, total=   2.0s
[CV] C=10, class_weight={1: 0.4, 0: 0.6}, penalty=l1, solver=liblinear 
[CV]  C=10, class_weight={1: 0.4, 0: 0.6}, penalty=l1, solver=liblinear, total=   2.1s
[CV] C=10, class_weight={1: 0.4, 0: 0.6}, penalty=l1, solver=saga ....
[CV]  C=10, class_weight={1: 0.4, 0: 0.6}, penalty=l1, solver=saga, total= 2.4min
[CV] C=10, class_weight={1: 0.4, 0: 0.6}, penalty=l1, solver=saga ....
[CV]  C=10, class_weight={1: 0.4, 0: 0.6}, penalty=l1, solver=saga, total= 2.4min
[CV] C=10, class_weight={1: 0.4, 0: 0.6}, penalty=l1, solver=saga ....
[CV]  C=10, class_weight={1: 0.4, 0: 0.6}, penalty=l1, solver=saga, total= 2.9min
[CV] C=10, class_weight={1: 0.4, 0: 0.6}, penalty=l1, solver=saga ....
[CV]  C=10, class_weight={1: 0.4, 0: 0.6}, penalty=l1, solver=saga, total= 3.5min
[CV] C=10, class_weight={1: 0.4, 0: 0.6}, penalty=l1, solver=saga ....
[CV]  C=10, class_weight={1: 0.4, 0: 0.6}, penalty=l1, solver=saga, tot

[CV]  C=10, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=liblinear, total=   2.0s
[CV] C=10, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=liblinear 
[CV]  C=10, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=liblinear, total=   2.1s
[CV] C=10, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=liblinear 
[CV]  C=10, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=liblinear, total=   2.2s
[CV] C=10, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=liblinear 
[CV]  C=10, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=liblinear, total=   3.7s
[CV] C=10, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=liblinear 
[CV]  C=10, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=liblinear, total=   2.0s
[CV] C=10, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=liblinear 
[CV]  C=10, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=liblinear, total=   2.0s
[CV] C=10, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=liblinear 
[CV]  C=10, class_weight={1: 0.6, 0: 0.4}, pen

[CV]  C=10, class_weight={1: 0.7, 0: 0.3}, penalty=l2, solver=saga, total=   7.6s
[CV] C=10, class_weight={1: 0.7, 0: 0.3}, penalty=l2, solver=saga ....
[CV]  C=10, class_weight={1: 0.7, 0: 0.3}, penalty=l2, solver=saga, total=   9.1s
[CV] C=10, class_weight={1: 0.7, 0: 0.3}, penalty=l2, solver=saga ....
[CV]  C=10, class_weight={1: 0.7, 0: 0.3}, penalty=l2, solver=saga, total=   8.5s
[CV] C=10, class_weight={1: 0.7, 0: 0.3}, penalty=l2, solver=saga ....
[CV]  C=10, class_weight={1: 0.7, 0: 0.3}, penalty=l2, solver=saga, total=   8.4s
[CV] C=10, class_weight={1: 0.7, 0: 0.3}, penalty=l2, solver=saga ....
[CV]  C=10, class_weight={1: 0.7, 0: 0.3}, penalty=l2, solver=saga, total=   9.2s
[CV] C=10, class_weight={1: 0.7, 0: 0.3}, penalty=l2, solver=saga ....
[CV]  C=10, class_weight={1: 0.7, 0: 0.3}, penalty=l2, solver=saga, total=   8.1s


[Parallel(n_jobs=1)]: Done 840 out of 840 | elapsed: 115.4min finished


Best Score:  0.9311986799197678
Best Params:  {'C': 10, 'class_weight': {1: 0.6, 0: 0.4}, 'penalty': 'l2', 'solver': 'liblinear'}


In [36]:
y = grid_logistic_combined_result.predict(train_vec_combined)
print("training f1 ",metrics.f1_score(train['class'],y)*100)
print("average f1 by cv 93.11986799197678")
y = grid_logistic_combined_result.predict(test_vec_combined)
print("testing f1",metrics.f1_score(test['class'],y)*100)


training f1  97.54650233938149
average f1 by cv 93.11986799197678
testing f1 93.3920704845815


# Random Forest

In [34]:
from rfpimp import permutation_importances

In [ ]:
from sklearn.base import clone 

def drop_col_feat_imp(model, X_train, y_train, random_state = 42):
    
    # clone the model to have the exact same specification as the one initially trained
    model_clone = clone(model)
    # set random_state for comparability
    model_clone.random_state = random_state
    # training and scoring the benchmark model
    model_clone.fit(X_train, y_train)
    benchmark_score = model_clone.score(X_train, y_train)
    # list for storing feature importances
    importances = []
    
    # iterating over all columns and storing feature importance (difference between benchmark and new model)
    for col in X_train.columns:
        model_clone = clone(model)
        model_clone.random_state = random_state
        model_clone.fit(X_train.drop(col, axis = 1), y_train)
        drop_col_score = model_clone.score(X_train.drop(col, axis = 1), y_train)
        importances.append(benchmark_score - drop_col_score)
    
    importances_df = imp_df(X_train.columns, importances)
    return importances_df


In [ ]:
from treeinterpreter import treeinterpreter as ti, utils

selected_rows = [31, 85]
selected_df = X_train.iloc[selected_rows,:].values
prediction, bias, contributions = ti.predict(rf, selected_df)

for i in range(len(selected_rows)):
    print("Row", selected_rows[i])
    print("Prediction:", prediction[i][0], 'Actual Value:', y_train[selected_rows[i]])
    print("Bias (trainset mean)", bias[i])
    print("Feature contributions:")
    for c, feature in sorted(zip(contributions[i], 
                                 X_train.columns), 
                             key=lambda x: -abs(x[0])):
        print(feature, round(c, 2))
    print("-"*20) 

<img src="https://miro.medium.com/max/1280/0*08KrYhXpVQdUXWrX" />

<img src="https://miro.medium.com/max/1280/1*6HVomcqW7BWuZ2vvGOEptw.png" />

In [37]:
param_grid_svm = {'C': [0.01,0.1,1, 10, 100], 'gamma': [1,0.1,0.01,0.001,0.0001],'kernel': ['linear']}

In [ ]:
#by default 5 fold cv
grid_svm = GridSearchCV(SVC(),param_grid_svm,refit=True,cv=10,verbose=2,n_jobs=-1)
grid_svm.fit(train_word_vec,train["class"])
print('Best Score: ', grid_svm.best_score_)
print('Best Params: ', grid_svm.best_params_)


Fitting 10 folds for each of 25 candidates, totalling 250 fits
[CV] C=0.01, gamma=1, kernel=linear ..................................


[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ................... C=0.01, gamma=1, kernel=linear, total= 2.6min
[CV] C=0.01, gamma=1, kernel=linear ..................................


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  2.6min remaining:    0.0s


[CV] ................... C=0.01, gamma=1, kernel=linear, total= 2.6min
[CV] C=0.01, gamma=1, kernel=linear ..................................
[CV] ................... C=0.01, gamma=1, kernel=linear, total= 2.6min
[CV] C=0.01, gamma=1, kernel=linear ..................................
[CV] ................... C=0.01, gamma=1, kernel=linear, total= 2.6min
[CV] C=0.01, gamma=1, kernel=linear ..................................
[CV] ................... C=0.01, gamma=1, kernel=linear, total= 2.5min
[CV] C=0.01, gamma=1, kernel=linear ..................................
[CV] ................... C=0.01, gamma=1, kernel=linear, total= 2.6min
[CV] C=0.01, gamma=1, kernel=linear ..................................
[CV] ................... C=0.01, gamma=1, kernel=linear, total= 2.7min
[CV] C=0.01, gamma=1, kernel=linear ..................................
[CV] ................... C=0.01, gamma=1, kernel=linear, total= 2.6min
[CV] C=0.01, gamma=1, kernel=linear ..................................
[CV] .

[CV] .................... C=0.1, gamma=1, kernel=linear, total= 2.6min
[CV] C=0.1, gamma=0.1, kernel=linear .................................
[CV] .................. C=0.1, gamma=0.1, kernel=linear, total= 2.6min
[CV] C=0.1, gamma=0.1, kernel=linear .................................
[CV] .................. C=0.1, gamma=0.1, kernel=linear, total= 2.4min
[CV] C=0.1, gamma=0.1, kernel=linear .................................
[CV] .................. C=0.1, gamma=0.1, kernel=linear, total= 2.7min
[CV] C=0.1, gamma=0.1, kernel=linear .................................
[CV] .................. C=0.1, gamma=0.1, kernel=linear, total= 2.6min
[CV] C=0.1, gamma=0.1, kernel=linear .................................
[CV] .................. C=0.1, gamma=0.1, kernel=linear, total= 2.6min
[CV] C=0.1, gamma=0.1, kernel=linear .................................
[CV] .................. C=0.1, gamma=0.1, kernel=linear, total= 2.4min
[CV] C=0.1, gamma=0.1, kernel=linear .................................
[CV] .

[CV] .................... C=1, gamma=0.1, kernel=linear, total= 2.8min
[CV] C=1, gamma=0.1, kernel=linear ...................................
[CV] .................... C=1, gamma=0.1, kernel=linear, total= 2.8min
[CV] C=1, gamma=0.1, kernel=linear ...................................
[CV] .................... C=1, gamma=0.1, kernel=linear, total= 2.7min
[CV] C=1, gamma=0.01, kernel=linear ..................................
[CV] ................... C=1, gamma=0.01, kernel=linear, total= 2.7min
[CV] C=1, gamma=0.01, kernel=linear ..................................
[CV] ................... C=1, gamma=0.01, kernel=linear, total= 2.7min
[CV] C=1, gamma=0.01, kernel=linear ..................................
[CV] ................... C=1, gamma=0.01, kernel=linear, total= 2.7min
[CV] C=1, gamma=0.01, kernel=linear ..................................
[CV] ................... C=1, gamma=0.01, kernel=linear, total= 2.7min
[CV] C=1, gamma=0.01, kernel=linear ..................................
[CV] .

[CV] .................. C=10, gamma=0.01, kernel=linear, total= 4.2min
[CV] C=10, gamma=0.01, kernel=linear .................................
[CV] .................. C=10, gamma=0.01, kernel=linear, total= 4.2min
[CV] C=10, gamma=0.01, kernel=linear .................................
[CV] .................. C=10, gamma=0.01, kernel=linear, total= 4.1min
[CV] C=10, gamma=0.01, kernel=linear .................................
[CV] .................. C=10, gamma=0.01, kernel=linear, total= 4.2min
[CV] C=10, gamma=0.01, kernel=linear .................................
[CV] .................. C=10, gamma=0.01, kernel=linear, total= 4.3min
[CV] C=10, gamma=0.001, kernel=linear ................................
[CV] ................. C=10, gamma=0.001, kernel=linear, total= 4.2min
[CV] C=10, gamma=0.001, kernel=linear ................................
[CV] ................. C=10, gamma=0.001, kernel=linear, total= 4.7min
[CV] C=10, gamma=0.001, kernel=linear ................................
[CV] .

In [334]:
pipe = Pipeline(
        [('select', SelectFromModel(LogisticRegression(class_weight='balanced',
                                                  penalty="l1", C=0.01))),
        ('model', LogisticRegression(class_weight='balanced',penalty='l2'))])

In [431]:
%run ../src/classifier/NBSVM

In [619]:
model = NbSvmClassifier(C=10)

In [530]:
predict = model.predict(hstack((test_vec,test_vec_char)).tocsr())

In [598]:
df3 = hstack((test_vec,test_vec_char,test_data[Columns])).tocsr()

In [531]:
metrics.f1_score(test_data['class'],predict)

0.9304790605840174

In [575]:
df3.shape

(11866, 30000)

In [576]:
from scipy.sparse import csr_matrix, hstack


In [597]:
#frame = [train_vec.toarray(),train_data[Columns]]
df2 = hstack((train_vec,train_vec_char,train_data[Columns])).tocsr()

In [529]:
model.fit(df2,train_data["class"])

NbSvmClassifier(C=10, dual=False, n_jobs=1)

In [483]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [544]:
scores = []
classifier = LogisticRegression(C=10, solver='sag')
cv_score = np.mean(cross_val_score(classifier, df2, train_data["class"] ,cv=10, scoring='f1'))
scores.append(cv_score)
print('CV score for class {} is {}'.format(class_name, cv_score))

   

CV score for class 0 is 0.931595865109229


In [545]:
classifier.fit(df2, train_data["class"])
print('Total CV score is {}'.format(np.mean(scores)))

Total CV score is 0.931595865109229


In [570]:
model = LogisticRegression(solver='sag')
sfm = SelectFromModel(model, threshold=0.2)
train_sparse_matrix = sfm.fit_transform(df2, train_data['class'])
test_sparse_matrix = sfm.transform(df3)
d_train = lgb.Dataset(train_sparse_matrix, label=train_data["class"])
d_valid = lgb.Dataset(test_sparse_matrix, label=test_data["class"])
watchlist = [d_train, d_valid]
params = {'learning_rate': 0.2,
             'application': 'binary',
              'num_leaves': 31,
              'verbosity': -1,
              'metric': 'f1',
              'data_random_seed': 2,
              'bagging_fraction': 0.8,
              'feature_fraction': 0.6,
              'nthread': 4,
              'lambda_l1': 1,
          'lambda_l2': 1}
model1 = lgb.train(params,
                      train_set=d_train,
                      verbose_eval=10)
    


In [552]:
import lightgbm as lgb



In [557]:
?lgb.train

In [579]:
p = model.predict(test_sparse_matrix)

In [625]:
metrics.f1_score(test_data['class'],p>0.42)

0.9315718856435132

In [615]:
select = SelectFromModel(LogisticRegression(class_weight='balanced',penalty="l1",C=10,solver='liblinear'))
X_ = select.fit_transform(df2,train_data["class"])

In [627]:
model = LinearSVC(class_weight='balanced',C=0.1, penalty='l2', loss='squared_hinge').fit(X_, train_data["class"])

In [628]:
p1 = model.predict(select.transform(df3))

In [629]:
metrics.f1_score(test_data['class'],p1)

0.9238144867118291